In [6]:
import os

In [7]:
%pwd

'd:\\'

In [9]:
os.chdir('d:\\Dibu\\Course_Notes\\End-to-End-MLflow-Project')

In [10]:
%pwd

'd:\\Dibu\\Course_Notes\\End-to-End-MLflow-Project'

In [14]:
import pandas as pd

df = pd.read_csv(r'D:\Dibu\Course_Notes\End-to-End-MLflow-Project\artifacts\data_ingestion\winequality-red.csv')

In [ ]:
#  Just want to check the data types are as per the schema we defined
#  In this project we are not performing EDA, so if we want to perform any EDA this is the place
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict    

In [ ]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories



class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [ ]:
import os
from mlProject import logger

# The below functions just checks the data type of the columns in the data
# and compares it with the schema we defined in the schema.yaml file
# This is a simple data validation step to ensure that the data types are as expected
# This is not a comprehensive data validation step, but it is a good starting point
# if the data types are not as expected, it will raise an error


class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [ ]:
try:
    config_manager = ConfigurationManager()
    data_validation_config = config_manager.get_data_validation_config()

    data_validation = DataValiadtion(config=data_validation_config)
    status = data_validation.validate_all_columns()

    logger.info(f"Data validation status: {status}")
    
except Exception as e:
    logger.exception(f"Exception occurred during data validation: {e}")
    raise e
